---
# **LAB 3 - Modello di esecuzione CUDA**
---

# ▶️ CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

GPU computing notebooks download (from github)

In [ ]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

NVCC Plugin for Jupyter notebook

In [ ]:
%cd GPUcomputing/utils/nvcc4jupyter-master/
!python3 setup.py install
%load_ext nvcc4jupyter
%cd /content/

# ✅ Divergence analysis

In [ ]:
%%cuda_group_save --name "div.cu" --group "DIV"
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include "/content/GPUcomputing/utils/common.h"

/*
 * Kernel with warp divergence
 */
__global__ void evenOddDIV(int *c, const ulong N) {
	ulong tid = blockIdx.x * blockDim.x + threadIdx.x;
	int a, b;

	if (!(tid % 2))   // branch divergence
		a = 2;
	else
		b = 1;

	// check index
	if (tid < N)
		c[tid] = a + b;
}

/*
 * MAIN
 */
int main(int argc, char **argv) {

	// set up data size
	int blocksize = 1024;
	ulong size = 1024*1024;

	if (argc > 1)
		blocksize = atoi(argv[1]);
	if (argc > 2)
		size = atoi(argv[2]);
	ulong nBytes = size * sizeof(int);

	printf("Data size: %lu  -- ", size);
  printf("Data size (bytes): %lu MB\n", nBytes/1000000);

	// set up execution configuration
	dim3 block(blocksize, 1);
	dim3 grid((size + block.x - 1) / block.x, 1);
	printf("Execution conf (block %d, grid %d)\nKernels:\n", block.x, grid.x);

	// allocate memory
	int *d_C, *C;
	C = (int *) malloc(nBytes);
	CHECK(cudaMalloc((void** )&d_C, nBytes));

	// run kernel 1
	double iStart, iElaps;
	iStart = seconds();
	evenOddDIV<<<grid, block>>>(d_C, size);
	CHECK(cudaDeviceSynchronize());
	iElaps = seconds() - iStart;
	printf("\tevenOddDIV<<<%d, %d>>> elapsed time %f sec \n\n", grid.x, block.x, iElaps);
	CHECK(cudaGetLastError());

  CHECK(cudaMemcpy(C, d_C, nBytes, cudaMemcpyDeviceToHost));

	free(C);
	// free gpu memory and reset device
	CHECK(cudaFree(d_C));
	CHECK(cudaDeviceReset());
	return EXIT_SUCCESS;
}


In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_75 src/DIV/div.cu -o div
!./div 1024 20000000

In [ ]:
!ncu  ./div

In [ ]:
# Compilazione ed esecuzione versione di debug
!nvcc -arch=sm_75 -g -G src/DIV/div.cu -o div_deb
!./div_deb 1024 2000000000

In [ ]:
!ncu ./div_deb

### ↘️ *`TODO...`*

Introdurre nuovo kernel che eviti la divergenza a livello di warp.

- usare/creare nuova indicizzazione a livello di warp
- applicare nuova indicizzazione preservando il risultato finale


In [ ]:
%%cuda_group_save --name "div.cu" --group "DIV"
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include "/content/GPUcomputing/utils/common.h"

/*
 * Kernel with warp divergence
 */
__global__ void evenOddDIV(int *c, const ulong N) {
	ulong tid = blockIdx.x * blockDim.x + threadIdx.x;
	int a, b;

	if (!(tid % 2))   // branch divergence
		a = 2;
	else
		b = 1;

	// check index
	if (tid < N)
		c[tid] = a + b;
}

/*
 * Kernel without warp divergence
 */
__global__ void evenOddNODIV(int *c, const int N) {

	// TODO

  // warp index wid = 0,1,2,3,...

	// using wid index for even and odd

	// check index

}

/*
 * MAIN
 */
int main(int argc, char **argv) {

	// set up data size
	int blocksize = 1024;
	ulong size = 1024*1024;

	if (argc > 1)
		blocksize = atoi(argv[1]);
	if (argc > 2)
		size = atoi(argv[2]);
	ulong nBytes = size * sizeof(int);

	printf("Data size: %lu  -- ", size);
  printf("Data size (bytes): %lu MB\n", nBytes/1000000);

	// set up execution configuration
	dim3 block(blocksize, 1);
	dim3 grid((size + block.x - 1) / block.x, 1);
	printf("Execution conf (block %d, grid %d)\nKernels:\n", block.x, grid.x);

	// allocate memory
	int *d_C, *C;
	C = (int *) malloc(nBytes);
	CHECK(cudaMalloc((void** )&d_C, nBytes));

	// run kernel 1
	double iStart, iElaps;
	iStart = seconds();
	evenOddDIV<<<grid, block>>>(d_C, size);
	CHECK(cudaDeviceSynchronize());
	iElaps = seconds() - iStart;
	printf("\tevenOddDIV<<<%d, %d>>> elapsed time %f sec \n\n", grid.x, block.x, iElaps);
	CHECK(cudaGetLastError());

  CHECK(cudaMemcpy(C, d_C, nBytes, cudaMemcpyDeviceToHost));

	// run kernel 2
  CHECK(cudaMemset(d_C, 0.0, nBytes)); // reset memory
	iStart = seconds();
	evenOddNODIV<<<grid, block>>>(d_C, size);
	iElaps = seconds() - iStart;
	printf("\tevenOddNODIV<<<%d, %d>>> elapsed time %f sec \n\n", grid.x, block.x, iElaps);
	CHECK(cudaGetLastError());

	CHECK(cudaMemcpy(C, d_C, nBytes, cudaMemcpyDeviceToHost));

	free(C);
	// free gpu memory and reset device
	CHECK(cudaFree(d_C));
	CHECK(cudaDeviceReset());
	return EXIT_SUCCESS;
}


In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_75 src/DIV/div.cu -o div
!./div 1024 20000000

# ✅ Parallel Reduction

In [ ]:
%%cuda_group_save --name "preduce.cu" --group "PAR"
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#include "/content/GPUcomputing/utils/common.h"


/*
 *  Block by block parallel implementation with divergence (sequential schema)
 */
__global__ void blockParReduce1(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = 1; stride < blockDim.x; stride *= 2) {
		if ((tid % (2 * stride)) == 0)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}



/*
 * MAIN: test on parallel reduction
 */
int main(void) {
	int *a, *b, *d_a, *d_b;
	int blockSize = 1024;            // block dim 1D
	ulong numBlock = 1024*1024;      // grid dim 1D
	ulong n = blockSize * numBlock;  // array dim
	long sum_CPU = 0, sum_GPU;
	long nByte = n*sizeof(int), mByte = numBlock * sizeof(int);
	double start, stopGPU, stopCPU, speedup;

	printf("\n****  test on parallel reduction  ****\n");

	// init
	a = (int *) malloc(nByte);
	b = (int *) malloc(mByte);
	for (ulong i = 0; i < n; i++) a[i] = 1;

	CHECK(cudaMalloc((void **) &d_a, nByte));
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void **) &d_b, mByte));
	CHECK(cudaMemset((void *) d_b, 0, mByte));

	/***********************************************************/
	/*                     CPU reduction                       */
	/***********************************************************/
	printf("  Vector length: %.2f MB\n",n/(1024.0*1024.0));
	printf("\n  CPU procedure...\n");
	start = seconds();
	for (ulong i = 0; i < n; i++)
    sum_CPU += a[i];
	stopCPU = seconds() - start;
	printf("    Elapsed time: %f (sec) \n", stopCPU);
	printf("    sum: %lu\n",sum_CPU);

	printf("\n  GPU kernels (mem required %lu bytes)\n", nByte);

	/***********************************************************/
	/*         KERNEL blockParReduce1 (divergent)              */
	/***********************************************************/
	// block by block parallel implementation with divergence
	printf("\n  Launch kernel: blockParReduce1...\n");
	start = seconds();
	blockParReduce1<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaGetLastError());
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);

  // memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

  // check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);

	// reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i]=1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));


	cudaFree(d_a);

	CHECK(cudaDeviceReset());
	return 0;
}


In [ ]:
#Compilazione ed esecuzione

!nvcc -arch=sm_75 src/PAR/preduce.cu -o preduce
!./preduce

### ↘️ *`TODO...`*

Kernel privo di divergenza:

* Usare lo schema che suddivide in blocchi (richiesta sincronizzazione)
* Sommare su ogni blocco con parallel reduction (somma parziale)
* Utilizzare uno schema interlacciato
* Evitare la divergenza nella parallel reduction
* Unire le somme parziali dei blocchi




In [ ]:
%%cuda_group_save --name "preduce.cu" --group "PAR"
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include "/content/GPUcomputing/utils/common.h"


/*
 *  Block by block parallel implementation with divergence (sequential schema)
 */
__global__ void blockParReduce1(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = 1; stride < blockDim.x; stride *= 2) {
		if ((tid % (2 * stride)) == 0)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
 *  Block by block parallel implementation without divergence (interleaved schema)
 */
__global__ void blockParReduce2(int *in, int *out, ulong n) {

	// TODO

	// boundary check

	// convert global data pointer to the local pointer of this block

	// in-place reduction in global memory

	// write result for this block to global mem

}


/*
 * MAIN: test on parallel reduction
 */
int main(void) {
	int *a, *b, *d_a, *d_b;
	int blockSize = 1024;            // block dim 1D
	ulong numBlock = 1024*1024;      // grid dim 1D
	ulong n = blockSize * numBlock;  // array dim
	long sum_CPU = 0, sum_GPU;
	long nByte = n*sizeof(int), mByte = numBlock * sizeof(int);
	double start, stopGPU, stopCPU, speedup;

	printf("\n****  test on parallel reduction  ****\n");

	// init
	a = (int *) malloc(nByte);
	b = (int *) malloc(mByte);
	for (ulong i = 0; i < n; i++) a[i] = 1;

	CHECK(cudaMalloc((void **) &d_a, nByte));
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void **) &d_b, mByte));
	CHECK(cudaMemset((void *) d_b, 0, mByte));

	/***********************************************************/
	/*                     CPU reduction                       */
	/***********************************************************/
	printf("  Vector length: %.2f MB\n",n/(1024.0*1024.0));
	printf("\n  CPU procedure...\n");
	start = seconds();
	for (ulong i = 0; i < n; i++)
    sum_CPU += a[i];
	stopCPU = seconds() - start;
	printf("    Elapsed time: %f (sec) \n", stopCPU);
	printf("    sum: %lu\n",sum_CPU);

	printf("\n  GPU kernels (mem required %lu bytes)\n", nByte);

	/***********************************************************/
	/*         KERNEL blockParReduce1 (divergent)              */
	/***********************************************************/
	// block by block parallel implementation with divergence
	printf("\n  Launch kernel: blockParReduce1...\n");
	start = seconds();
	blockParReduce1<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaGetLastError());
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);

  // memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

  // check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);

	// reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i]=1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*        KERNEL blockParReduce2  (non divergent)          */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: blockParReduce2...\n");
	start = seconds();
	blockParReduce2<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());

  // memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

  // check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
  //		printf("b[%d] = %d\n",i,b[i]);
	}
	assert(sum_GPU == n);

  // reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);

	cudaFree(d_a);

	CHECK(cudaDeviceReset());
	return 0;
}


In [ ]:
#Compilazione ed esecuzione

!nvcc -arch=sm_75 src/PAR/preduce.cu -o preduce
!./preduce

# ✅ Istogramma di un'immagine BMP

Calcolare l'istogramma di un aimmagine BMP con uso di `atomicAdd`

↘️ **TODO...**

In [ ]:
%%cuda_group_save --name "ppm_hist.cu" --group "lez3"

%%cuda_group_save --name "ppm_hist.cu" --group "lez3"


#include <stdio.h>
#include <time.h>
#include <limits.h>

#include "/content/GPUcomputing/utils/common.h"
#include "ppm.h"

/*
 * Kernel 1D that computes histogram on GPU
 */
__global__ void ppm_histGPU(PPM ppm, int *histogram) {

   // TODO

}

/*
 * MAIN
 */
int main(int argc, char **argv) {

   // PPM images
   PPM *ppm;   // Where images are stored in CPU
   PPM ppm_d;  // Where images are stored in GPU

   // load a PPM image from file
   char path[] = "GPUcomputing/images/dog.ppm";
   ppm = ppm_load(path);
   uint WIDTH = ppm->width;
   uint HEIGHT = ppm->height;
   ppm_d.width = WIDTH;
   ppm_d.height = HEIGHT;
   ppm_d.maxval = ppm->maxval;
   printf("PPM image size (w x h): %d x %d\n", WIDTH, HEIGHT);

   // set main params
   uint nBins = 3 * 256 * sizeof(int);
   int nBytes = WIDTH * HEIGHT * sizeof(int);

   // CPU version
   double start = seconds();
   int *histogram = ppm_histogram(ppm);
   double stopCPU = seconds() - start;

   // Allocate GPU buffer & copy image from CPU to GPU
   CHECK(cudaMalloc(&ppm_d.image, nBytes));
   CHECK(cudaMemcpy(ppm_d.image, ppm->image, nBytes, cudaMemcpyHostToDevice));

   // allocate memory for the histogram on the device
   int *histogram_d;
   CHECK(cudaMalloc(&histogram_d, nBins));
   CHECK(cudaMemset(histogram_d, 0, nBins));

   // invoke kernel
   dim3 dimBlock(512);
   dim3 dimGrid((WIDTH * HEIGHT + dimBlock.x - 1) / dimBlock.x);
   start = seconds();
   ppm_histGPU<<<dimGrid, dimBlock>>>(ppm_d, histogram_d);
   CHECK(cudaDeviceSynchronize());
   double stopGPU = seconds() - start;

   // read histogram from device memory
   int *histogram1 = (int*)malloc(nBins);  // for GPU copy to CPU
   CHECK(cudaMemcpy(histogram1, histogram_d, nBins, cudaMemcpyDeviceToHost));
   ppm_save_histogram(histogram1, "histogramGPU.ppm");
   ppm_save_histogram(histogram, "histogramCPU.ppm");

   // check results
   for (int i = 0; i < 256; i++) {
      if (histogram[i] != histogram1[i]) {
         printf("Error r: hist[%d] = %d != d_hist[%d] = %d\n", i, histogram[i], i, histogram1[i]);
         break;
      }
      if (histogram[i+256] != histogram1[i+256]) {
         printf("Error g: hist[%d] = %d != d_hist[%d] = %d\n", i, histogram[i], i, histogram1[i]);
         break;
      }
      if (histogram[i+512] != histogram1[i+512]) {
         printf("Error b: hist[%d] = %d != d_hist[%d] = %d\n", i, histogram[i], i, histogram1[i]);
         break;
      }
   }
   printf("Test PASSED\n");

   // times & speedup
   printf("CPU elapsed time: %.4f (msec) \n", stopCPU*1000);
   printf("CPU elapsed time: %.4f (msec) - Speedup %.1f\n", stopGPU*1000, stopCPU/stopGPU);

   // free memory
   free(histogram);
   CHECK(cudaFree(histogram_d));
   cudaFree(ppm_d.image);
   return 0;
}

}

↩ **Run...**

In [ ]:
!nvcc -arch=sm_75 src/lez3/ppm_hist.cu -o ppm_hist -I GPUcomputing/utils/PPM GPUcomputing/utils/PPM/ppm.cpp
!./ppm_hist

# ✅ Prodotto MQDB CUDA

### ↘️ *`TODO...`*

Calcolare il prodotto di matrici MQDB con kernel CUDA

In [ ]:
%%cuda_group_save --name "mqdb_prod.cu" --group "MQDB"

#include "/content/GPUcomputing/utils/common.h"
#include "/content/GPUcomputing/utils/MQDB/mqdb.h"

#define BLOCK_SIZE 16     // block size

struct tms {
	double CPUtms;
	double GPUtmsNaive;
	double GPUtmsMQDB;
	float density;
};

/*
 * Kernel for standard (naive) matrix product
 */
__global__ void matProd(mqdb A, mqdb B, mqdb C, int n) {
	// row & col indexes
	int row = blockIdx.y * blockDim.y + threadIdx.y;
	int col = blockIdx.x * blockDim.x + threadIdx.x;

	// each thread computes an entry of the product matrix
	if ((row < n) && (col < n)) {
		float val = 0;
		for (int k = 0; k < n; k++)
			val += A.elem[row * n + k] * B.elem[k * n + col];
		C.elem[row * n + col] = val;
	}
}

/*
 * Kernel for block sub-matrix product of mqdb
 */
__global__ void mqdbBlockProd(mqdb A, mqdb B, mqdb C, int sdim, int d, int n) {

	// TODO

	// jump to the right block sub-matrix

	// each thread computes an entry of the product matrix
}

/*
 * Test on MQDB kernels
 */
void testKernelsMQDB(uint n, uint k, struct tms* times) {

	// mqdb host matrices
	mqdb A, B, C, C1;

	// mqdb device matrices
	mqdb d_A, d_B, d_C;

	// fill in
	A = mqdbConst(n, k, 10, 1);
	B = mqdbConst(n, k, 10, 1);
	C = mqdbConst(n, k, 10, 1);
	C1 = mqdbConst(n, k, 10, 1);

	ulong nBytes = n * n * sizeof(float);
	ulong kBytes = k * sizeof(uint);
	printf("        Memory size required = %.1f (MB)\n",(float)nBytes/(1024.0*1024.0));

	// malloc and copy on device memory
	d_A.nBlocks = A.nBlocks;
	CHECK(cudaMalloc((void**)&d_A.blkSize, kBytes));
	CHECK(cudaMemcpy(d_A.blkSize, A.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_A.elem, nBytes));
	CHECK(cudaMemcpy(d_A.elem, A.elem, nBytes, cudaMemcpyHostToDevice));
	d_B.nBlocks = B.nBlocks;
	CHECK(cudaMalloc((void**)&d_B.blkSize, kBytes));
	CHECK(cudaMemcpy(d_B.blkSize, B.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_B.elem, nBytes));
	CHECK(cudaMemcpy(d_B.elem, B.elem, nBytes, cudaMemcpyHostToDevice));
	d_C.nBlocks = C.nBlocks;
	CHECK(cudaMalloc((void**)&d_C.blkSize, kBytes));
	CHECK(cudaMemcpy(d_C.blkSize, C.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_C.elem, nBytes));
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*                    CPU MQDB product                     */
	/***********************************************************/
	printf("\nCPU MQDB product...\n");
	double start = seconds();
	mqdbProd(A,B,C);
	double CPUTime = seconds() - start;
	printf("   CPU elapsed time: %.5f (sec)\n\n", CPUTime);

	/***********************************************************/
	/*                     GPU mat product                     */
	/***********************************************************/
	printf("Kernel (naive) mat product...\n");
	dim3 block(BLOCK_SIZE, BLOCK_SIZE);
	dim3 grid((n + block.x - 1) / block.x, (n + block.y - 1) / block.y);
	start = seconds();
	matProd<<<grid, block>>>(d_A, d_B, d_C, n);
	CHECK(cudaDeviceSynchronize());
	double GPUtime1 = seconds() - start;
	printf("   elapsed time:                %.4f (sec)\n", GPUtime1);
	printf("   speedup vs CPU MQDB product: %.4f\n", CPUTime/GPUtime1);
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));
	checkResult(C,C1);
	//	mqdbDisplay(C1);

	/***********************************************************/
	/*                     GPU MQDB product                    */
	/***********************************************************/
	printf("Kernel MQDB product...\n");
	double start = seconds();

	// TODO

 	CHECK(cudaDeviceSynchronize());
	double GPUtime2 = seconds() - start;
	printf("   elapsed time:                    %.4f (sec)\n", GPUtime2);
	printf("   speedup vs CPU MQDB product:     %.4f\n", CPUTime/GPUtime2);
	printf("   speedup vs GPU std mat product:  %.4f\n", GPUtime1/GPUtime2);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));
	checkResult(C,C1);

	CHECK(cudaFree(d_A.elem));
	CHECK(cudaFree(d_B.elem));
	CHECK(cudaFree(d_C.elem));

	// collect times
	times->CPUtms = CPUTime;
	times->GPUtmsNaive = GPUtime1;
	times->GPUtmsMQDB = GPUtime2;

	float den = 0;
	for (uint j = 0; j < k; j++)
		den += A.blkSize[j]*A.blkSize[j];
	times->density = den/(n*n);
}

/*
 * main function
 */
int main(int argc, char *argv[]) {
	uint n = 8*1024;      // matrix size
	uint min_k = 30;       // max num of blocks
	uint max_k = 30;       // max num of blocks

	struct tms times[max_k-min_k+1];

	// multiple tests on kernels
	for (uint k = min_k; k <= max_k; k++) {
		printf("\n*****   k = %d --- (avg block size = %f)\n",k,(float)n/k);
		testKernelsMQDB(n, k, &times[k-min_k]);
	}

	FILE *fd;
	fd = fopen("res.csv", "w");
	if (fd == NULL) {
		perror("file error!\n");
		exit(1);
	}

	// write results on file
	fprintf(fd,"num blocks,");
		for (uint j = 0; j <= max_k-min_k; j++)
			fprintf(fd,"%d,",j+min_k);

	fprintf(fd,"\nCPU MQDB product,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.6f,",times[j].CPUtms);

	fprintf(fd,"\nKernel mat product naive,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.6f,",times[j].GPUtmsNaive);

	fprintf(fd,"\nKernel MQDB product,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.6f,",times[j].GPUtmsMQDB);

	fprintf(fd,"\ndensity,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.6f,",times[j].density);

	fclose(fd);

	return 0;
}



In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75 src/MQDB/mqdb_prod.cu /content/GPUcomputing/utils/MQDB/mqdb.cpp  -o mqdb_prod
!./mqdb_prod